In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random
import tensorflow as tf

from skimage import color, exposure, filters, io, morphology, util
from math import sqrt
from numpy import loadtxt
from tensorflow import keras
from keras import backend as K
from keras.models import Sequential
from keras.layers import Conv2D, Dense, Dropout, Flatten, MaxPooling2D, Reshape
from keras.optimizers import SGD

### A. _Fully Connected Neural Networks_

1. Lakukan klasifikasi untuk dataset cifar menggunakan **_Fully Connected Neural Networks_** dengan arsitektur dasar sebagai berikut:
    - _Fully Connected Layer_ ($1000$ _hidden node_).
    - _Activation Layer_ ($\text{ReLU function}$).
    - _Fully Connected Layer_ ($1000$ _hidden node_)
    - _Activation Layer_ ($\text{ReLU function}$).
    - _Output Layer_ (10 kelas).
    - _Activation Layer_ ($\text{Softmax function}$).
    - _Classification Result_.

Fitur yang digunakan bebas (_handcrafted_/_non-convolution_). Hitung metrik ($\text{accuracy}$, $\text{sensitivity}$, $\text{specificity}$, dan $f_{1}\text{ score}$) untuk data train dan data test.

#### Mendefinisikan Fungsi Bantuan
Untuk memudahkan implementasi, berikut beberapa fungsi bantuan yang akan dipakai

##### Menghitung $\text{recall}$ atau $\text{sensitivity}$
$$\text{recall} = \frac{t_p}{t_p+t_n}$$

In [ ]:
def recall(y_true, y_pred): 
  true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
  possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
  return true_positives / (possible_positives + K.epsilon())

##### Menghitung $\text{precision}$
$$\text{precision} = \frac{t_p}{t_p + f_p}$$

In [ ]:
def precision(y_true, y_pred):
  true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
  predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
  return true_positives / (predicted_positives + K.epsilon())

##### Menghitung $\text{specificity}$
$$\text{specificity} = \frac{t_n}{t_n + f_p}$$

In [ ]:
def specificity(y_true, y_pred):
  true_negatives = K.sum(K.round(K.clip((1 - y_true) * (1 - y_pred), 0, 1)))
  possible_negatives = K.sum(K.round(K.clip(1 - y_true, 0, 1)))
  return true_negatives / (possible_negatives + K.epsilon())

##### Menghitung $f_{1}\text{ Score}$
Kita tau bahwa
$$f_{\beta} = (1 + \beta^{2}) \cdot \frac{\text{precition} \cdot \text{recall}}{(\beta^{2} \cdot \text{precision}) + \text{recall}}$$
Sehingga 
$$f_{1} = (1 + 1^{2}) \cdot \frac{\text{precition} \cdot \text{recall}}{(1^{2} \cdot \text{precision}) + \text{recall}}$$
$$f_{1} = (2) \cdot \frac{\text{precition} \cdot \text{recall}}{(\text{precision}) + \text{recall}}$$



In [ ]:
def f1_score(y_true, y_pred):
  precision_val = precision(y_true, y_pred)
  recall_val = recall(y_true, y_pred)
  return 2*((precision_val * recall_val)/(precision_val + recall_val + K.epsilon()))

**n.b.: Semua fungsi pada bagian penyebut ditambah epsilon untuk menghindari dibagi $0$ dikarenakan memasukkan input sebanyak $0$**

##### Fungsi Bantuan untuk Melakukan Prediksi dan Menampilkan Metric

In [ ]:
def predict_and_get_metrics(model, x_test, y_test, label):
  _, accuracy, sensitivity, specificity, score_f1 = model.evaluate(x_test, y_test, verbose=0)
  return pd.DataFrame([[accuracy, sensitivity, specificity, score_f1]], columns=['Accuracy', 'Sensitivity', 'Specificity', 'F1 Score'], index=[label])

def compare_train_and_test(model, x_train, y_train, x_test, y_test, extra_label=''):
  metric_train = predict_and_get_metrics(model, x_train, y_train, 'Data Train %s' % (extra_label))
  metric_test = predict_and_get_metrics(model, x_test, y_test, 'Data Test %s' % (extra_label))
  return pd.concat([metric_train, metric_test])

##### Fungsi Bantuan untuk _Load Dataset_

In [ ]:
def load_dataset():
	(trainX, trainY), (testX, testY) = keras.datasets.cifar10.load_data()
	trainY = tf.keras.utils.to_categorical(trainY) ## Di ubah ke matrix persegi representasi binari
	testY = tf.keras.utils.to_categorical(testY) ## Di ubah ke matrix persegi representasi binari
	return trainX, trainY, testX, testY

#### Load Dataset

In [ ]:
x_train, y_train, x_test, y_test = load_dataset()

#### Membangun Model _Fully Connected Neural Network_ dengan spesifikasi minimum memenuhi deskripsi Lab

In [ ]:
## Melakukan inisialisasi node per layer
nodes_per_layer = [1000, 1000, 10]
activation_function_per_layer = [tf.nn.relu, tf.nn.relu, tf.nn.softmax]

## Inisialisasi Layer Model Parameter
layers = []
layers.append(tf.keras.layers.Flatten())

## Menyisipkan parameter node dan fungsi aktivasinya
for i in range(len(nodes_per_layer)):
  node = nodes_per_layer[i]
  activation = activation_function_per_layer[i]
  layers.append(keras.layers.Dense(node, activation=activation))

## Mendefinisikan model
model = Sequential(layers)

## Membangun model
model.build(input_shape=x_train.shape)
model.output_shape

(50000, 10)

#### Melakukan kompilasi ke dalam model

In [ ]:
model.compile(
    optimizer=SGD(lr=0.001, momentum=0.9),
    loss='categorical_crossentropy',
    metrics=[
      'accuracy',
      recall,
      specificity,
      f1_score
    ])

#### Melakukan Fitting

In [ ]:
beginning_hist = model.fit(x_train, y_train, epochs=10, batch_size=5000, validation_data=(x_test,y_test), use_multiprocessing=True)

Epoch 1/10
10/10 [==============================] - 16s 2s/step - loss: 1054907.8750 - accuracy: 0.0991 - recall: 0.0991 - specificity: 0.8999 - f1_score: 0.0991 - val_loss: 2450754.0000 - val_accuracy: 0.1000 - val_recall: 0.1000 - val_specificity: 0.9000 - val_f1_score: 0.1000
Epoch 2/10
10/10 [==============================] - 15s 2s/step - loss: 400731.0938 - accuracy: 0.0983 - recall: 0.0524 - specificity: 0.9468 - f1_score: 0.0534 - val_loss: 2.3237 - val_accuracy: 0.1000 - val_recall: 0.0000e+00 - val_specificity: 1.0000 - val_f1_score: 0.0000e+00
Epoch 3/10
10/10 [==============================] - 15s 2s/step - loss: 2.3440 - accuracy: 0.1000 - recall: 4.0000e-05 - specificity: 1.0000 - f1_score: 7.9960e-05 - val_loss: 2.3188 - val_accuracy: 0.1000 - val_recall: 0.0000e+00 - val_specificity: 1.0000 - val_f1_score: 0.0000e+00
Epoch 4/10
10/10 [==============================] - 15s 2s/step - loss: 2.3051 - accuracy: 0.1000 - recall: 0.0000e+00 - specificity: 1.0000 - f1_score: 0.

#### Melakukan Prediksi ke _Data Test_ dan Menampilkan _Metric_-nya

In [ ]:
compare_train_and_test(model, x_train, y_train, x_test, y_test).transpose()

,Data Train,Data Test
Accuracy,0.1,0.100000
Sensitivity,0.0,0.000000
Specificity,1.0,0.999989
F1 Score,0.0,0.000000


2. Anda dapat memperbaiki hasil yang diperoleh dengan melakukan  (tiga) skenario eksperimen, yaitu: _feature engineering_, menambahkan _layer_, dan _hyperparameter tuning_ (pada classifier).

#### _Feature Engineering_

- Untuk _feature engineering_ pada sesi lab kali ini, akan dilakukan normalisasi input agar berada pada rentang $[0...1]$.

In [ ]:
def normalize(x):
  return x / 255.0

normalize_x_train = normalize(x_train)
normalize_x_test = normalize(x_test)

##### Membangun Model _Fully Connected Neural Network_ sebagai _dedicated model_ untuk eksperimen _Feature Engineering_

In [ ]:
## Melakukan inisialisasi node per layer
nodes_per_layer_for_feature_engineering = [1000, 1000, 10]
activation_function_per_layer_for_feature_engineering = [tf.nn.relu, tf.nn.relu, tf.nn.softmax]

## Inisialisasi Layer Model Parameter
layers_for_feature_engineering = []
layers_for_feature_engineering.append(tf.keras.layers.Flatten())

## Menyisipkan parameter node dan fungsi aktivasinya
for i in range(len(nodes_per_layer_for_feature_engineering)):
  node = nodes_per_layer_for_feature_engineering[i]
  activation = activation_function_per_layer_for_feature_engineering[i]
  layers_for_feature_engineering.append(keras.layers.Dense(node, activation=activation))

## Mendefinisikan model
model_for_feature_engineering = Sequential(layers_for_feature_engineering)

## Membangun model
model_for_feature_engineering.build(input_shape=x_train.shape)
model_for_feature_engineering.output_shape

(50000, 10)

##### Melakukan kompilasi ke dalam model

In [ ]:
model_for_feature_engineering.compile(
    optimizer=SGD(lr=0.001, momentum=0.9),
    loss='categorical_crossentropy',
    metrics=[
      'accuracy',
      recall,
      specificity,
      f1_score
    ])

##### Melakukan _fitting_

In [ ]:
feature_engineering_hist = model_for_feature_engineering.fit(normalize_x_train, y_train, epochs=10, batch_size=5000, validation_data=(normalize_x_test, y_test), use_multiprocessing=True)

Epoch 1/10
10/10 [==============================] - 16s 2s/step - loss: 2.3174 - accuracy: 0.1373 - recall: 0.0000e+00 - specificity: 1.0000 - f1_score: 0.0000e+00 - val_loss: 2.2718 - val_accuracy: 0.1444 - val_recall: 0.0000e+00 - val_specificity: 1.0000 - val_f1_score: 0.0000e+00
Epoch 2/10
10/10 [==============================] - 15s 2s/step - loss: 2.2522 - accuracy: 0.1684 - recall: 0.0000e+00 - specificity: 1.0000 - f1_score: 0.0000e+00 - val_loss: 2.2108 - val_accuracy: 0.2226 - val_recall: 0.0000e+00 - val_specificity: 1.0000 - val_f1_score: 0.0000e+00
Epoch 3/10
10/10 [==============================] - 15s 2s/step - loss: 2.1918 - accuracy: 0.2126 - recall: 0.0000e+00 - specificity: 1.0000 - f1_score: 0.0000e+00 - val_loss: 2.1632 - val_accuracy: 0.2283 - val_recall: 0.0000e+00 - val_specificity: 1.0000 - val_f1_score: 0.0000e+00
Epoch 4/10
10/10 [==============================] - 15s 2s/step - loss: 2.1463 - accuracy: 0.2538 - recall: 0.0000e+00 - specificity: 1.0000 - f1_sc

#### Melakukan Prediksi ke _Data Test_ dan Menampilkan _Metric_-nya

In [ ]:
compare_train_and_test(model_for_feature_engineering, normalize_x_train, y_train, normalize_x_test, y_test).transpose()

,Data Train,Data Test
Accuracy,0.317200,0.316200
Sensitivity,0.004299,0.004493
Specificity,0.999801,0.999844
F1 Score,0.008296,0.008696


#### Menambahkan _Layer_

- Untuk eksperimen menambahkan _layer_ pada sesi laboratorium kali ini, saya akan menambahkan layer dua buah _hidden layer_ lagi dengan jumlah _node_ sama dengan _hidden layer_ yang lain.

##### Membangun Model _Fully Connected Neural Network_ sebagai _dedicated model_ untuk eksperimen Menambahkan _Layer_

In [ ]:
## Melakukan inisialisasi node per layer
nodes_per_new_layer = [1000, 1000, 1000, 1000, 10]
activation_function_per_new_layer = [tf.nn.relu, tf.nn.relu, tf.nn.relu, tf.nn.relu, tf.nn.softmax]

## Inisialisasi Layer Model Parameter
new_layers = []
new_layers.append(tf.keras.layers.Flatten())

## Menyisipkan parameter node dan fungsi aktivasinya
for i in range(len(nodes_per_new_layer)):
  node = nodes_per_new_layer[i]
  activation = activation_function_per_new_layer[i]
  new_layers.append(keras.layers.Dense(node, activation=activation))

## Mendefinisikan model
new_model = Sequential(new_layers)

## Membangun model
new_model.build(input_shape=x_train.shape)
new_model.output_shape

(50000, 10)

##### Melakukan kompilasi ke dalam model

In [ ]:
new_model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss='categorical_crossentropy',
    metrics=[
      'accuracy',
      recall,
      specificity,
      f1_score
    ])

##### Melakukan _fitting_

In [ ]:
new_model_hist = new_model.fit(x_train, y_train, epochs=10, batch_size=5000, validation_data=(x_test, y_test), use_multiprocessing=True)

Epoch 1/10
10/10 [==============================] - 25s 2s/step - loss: 650.1035 - accuracy: 0.1079 - recall: 0.1079 - specificity: 0.9009 - f1_score: 0.1079 - val_loss: 84.4537 - val_accuracy: 0.1000 - val_recall: 0.1000 - val_specificity: 0.9000 - val_f1_score: 0.1000
Epoch 2/10
10/10 [==============================] - 25s 2s/step - loss: 46.4477 - accuracy: 0.1037 - recall: 0.1032 - specificity: 0.9007 - f1_score: 0.1034 - val_loss: 18.1942 - val_accuracy: 0.1002 - val_recall: 0.1000 - val_specificity: 0.9002 - val_f1_score: 0.1001
Epoch 3/10
10/10 [==============================] - 25s 2s/step - loss: 8.3422 - accuracy: 0.1106 - recall: 0.0953 - specificity: 0.9175 - f1_score: 0.1030 - val_loss: 5.9730 - val_accuracy: 0.1267 - val_recall: 0.0902 - val_specificity: 0.9442 - val_f1_score: 0.1133
Epoch 4/10
10/10 [==============================] - 25s 2s/step - loss: 4.3884 - accuracy: 0.1356 - recall: 0.0680 - specificity: 0.9584 - f1_score: 0.0938 - val_loss: 2.9894 - val_accuracy: 

##### Melakukan Prediksi ke _Data Test_ dan Menampilkan _Metric_-nya

In [ ]:
compare_train_and_test(new_model, x_train, y_train, x_test, y_test).transpose()

,Data Train,Data Test
Accuracy,0.340520,0.336800
Sensitivity,0.069078,0.067392
Specificity,0.994668,0.994574
F1 Score,0.120937,0.118002


#### _Hyperparameter Tuning_

- Untuk eksperimen _hyperparameter tuning_ pada sesi laboratorium kali ini, saya akan membagi eksperimen menjadi:
    1. Meningkatkan epoch dari $10$ menjadi $30$.
    2. Menaikkan $\text{learning rate}$ dari $0.001$ menjadi $0.005$.
    3. Menurunkan $\text{learning rate}$ dari $0.001$ menjadi $0.0001$.
    4. Meningkatkan epoch dari $10$ menjadi $30$ dan menaikkan $\text{learning rate}$ dari $0.001$ menjadi $0.005$.
    5. Meningkatkan epoch dari $10$ menjadi $30$ dan menurunkan $\text{learning rate}$ dari $0.001$ menjadi $0.0001$.

- Membangun Model _Fully Connected Neural Network_ sebagai _dedicated model_ untuk eksperimen _Hyperparameter Tuning_

In [ ]:
## Melakukan inisialisasi node per layer
nodes_per_layer_for_hyperparameter = [1000, 1000, 10]
activation_function_per_layer_for_hyperparameter = [tf.nn.relu, tf.nn.relu, tf.nn.softmax]

## Inisialisasi Layer Model Parameter
layers_for_hyperparameter = []
layers_for_hyperparameter.append(tf.keras.layers.Flatten())

## Menyisipkan parameter node dan fungsi aktivasinya
for i in range(len(nodes_per_layer_for_hyperparameter)):
  node = nodes_per_layer_for_hyperparameter[i]
  activation = activation_function_per_layer_for_hyperparameter[i]
  layers_for_hyperparameter.append(keras.layers.Dense(node, activation=activation))

## Mendefinisikan model
model_for_hyperparameter = [
  Sequential(layers_for_hyperparameter)
  for _ in range(5) # 5 Mini Scenario
]

## Membangun model
for i in range(5): # 5 Mini Scenario
  model_for_hyperparameter[i].build(input_shape=x_train.shape)

- Melakukan kompilasi ke dalam model

In [ ]:
parameter_learning_setiap_skenario = [0.001, 0.005, 0.0001, 0.005, 0.0001]
for i in range(5):
  learning_rate = parameter_learning_setiap_skenario[i]
  model_for_hyperparameter[i].compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate),
    loss='categorical_crossentropy',
    metrics=[
      'accuracy',
      recall,
      specificity,
      f1_score
    ]      
  )

model_for_hyperparameter_hist = np.zeros(5, dtype=object)

- Melakukan _fitting_

In [ ]:
epochs = [30, 10, 10, 30, 30]
for i in range(5):
  model_for_hyperparameter_hist[i] = model_for_hyperparameter[i].fit(x_train, y_train, epochs=epochs[i], batch_size=5000, validation_data=(x_test, y_test), use_multiprocessing=True)

Epoch 1/30
10/10 [==============================] - 15s 2s/step - loss: 122907.3516 - accuracy: 0.0993 - recall: 0.0993 - specificity: 0.8999 - f1_score: 0.0993 - val_loss: 740560.5625 - val_accuracy: 0.1000 - val_recall: 0.1000 - val_specificity: 0.9000 - val_f1_score: 0.1000
Epoch 2/30
10/10 [==============================] - 15s 1s/step - loss: 92653611489689600.0000 - accuracy: 0.0996 - recall: 0.0996 - specificity: 0.9000 - f1_score: 0.0996 - val_loss: 81669894123852660736.0000 - val_accuracy: 0.1000 - val_recall: 0.1000 - val_specificity: 0.9000 - val_f1_score: 0.1000
Epoch 3/30
10/10 [==============================] - 15s 1s/step - loss: 8314836782730444800.0000 - accuracy: 0.0994 - recall: 0.0994 - specificity: 0.8999 - f1_score: 0.0994 - val_loss: 9464814.0000 - val_accuracy: 0.1000 - val_recall: 0.1000 - val_specificity: 0.9000 - val_f1_score: 0.1000
Epoch 4/30
10/10 [==============================] - 15s 1s/step - loss: 955383.8750 - accuracy: 0.0996 - recall: 0.0996 - speci

- Melakukan Prediksi ke _Data Test_ dan Menampilkan _Metric_-nya

In [ ]:
val = []
for i in range(5):
  val.append(compare_train_and_test(model_for_hyperparameter[i], x_train, y_train, x_test, y_test, str(i+1)))
pd.concat(val).transpose()

,Data Train 1,Data Test 1,Data Train 2,Data Test 2,Data Train 3,Data Test 3,Data Train 4,Data Test 4,Data Train 5,Data Test 5
Accuracy,0.100020,0.1,0.100020,0.1,0.100020,0.1,0.100020,0.1,0.100020,0.1
Sensitivity,0.000020,0.0,0.000020,0.0,0.000020,0.0,0.000020,0.0,0.000020,0.0
Specificity,1.000000,1.0,1.000000,1.0,1.000000,1.0,1.000000,1.0,1.000000,1.0
F1 Score,0.000039,0.0,0.000039,0.0,0.000039,0.0,0.000039,0.0,0.000039,0.0


#### Gabungan _Feature Engineering_, Menambahkan _Layer_, dan _Hyperparameter tuning_ ($\text{learning rate} = 0.001$, $\text{epoch} = 30$)

- Membangun Model _Fully Connected Neural Network_ sebagai _dedicated model_ untuk eksperimen _multiple parameter_.

In [ ]:
## Melakukan inisialisasi node per layer
nodes_per_layer_for_combined_trick = [1000, 1000, 1000, 1000, 10]
activation_function_per_layer_for_combined_trick = [tf.nn.relu, tf.nn.relu, tf.nn.relu, tf.nn.relu, tf.nn.softmax]

## Inisialisasi Layer Model Parameter
layers_for_combined_trick = []
layers_for_combined_trick.append(tf.keras.layers.Flatten())

## Menyisipkan parameter node dan fungsi aktivasinya
for i in range(len(nodes_per_layer_for_combined_trick)):
  node = nodes_per_layer_for_combined_trick[i]
  activation = activation_function_per_layer_for_combined_trick[i]
  layers_for_combined_trick.append(keras.layers.Dense(node, activation=activation))

## Mendefinisikan model
model_for_combined_trick = Sequential(layers_for_combined_trick)

## Membangun model
model_for_combined_trick.build(input_shape=x_train.shape)
model_for_combined_trick.output_shape

(50000, 10)

- Melakukan kompilasi ke dalam model

In [ ]:
model_for_combined_trick.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss='categorical_crossentropy',
    metrics=[
      'accuracy',
      recall,
      specificity,
      f1_score
    ])

- Melakukan _fitting_

In [ ]:
model_for_combined_trick_hist = model_for_combined_trick.fit(normalize_x_train, y_train, epochs=30, batch_size=5000, validation_data=(normalize_x_test, y_test), use_multiprocessing=True)

Epoch 1/30
10/10 [==============================] - 26s 3s/step - loss: 2.6113 - accuracy: 0.1169 - recall: 0.0066 - specificity: 0.9957 - f1_score: 0.0093 - val_loss: 2.2036 - val_accuracy: 0.1675 - val_recall: 0.0000e+00 - val_specificity: 1.0000 - val_f1_score: 0.0000e+00
Epoch 2/30
10/10 [==============================] - 25s 2s/step - loss: 2.1889 - accuracy: 0.1732 - recall: 0.0000e+00 - specificity: 1.0000 - f1_score: 0.0000e+00 - val_loss: 2.1035 - val_accuracy: 0.2131 - val_recall: 0.0000e+00 - val_specificity: 1.0000 - val_f1_score: 0.0000e+00
Epoch 3/30
10/10 [==============================] - 24s 2s/step - loss: 2.0780 - accuracy: 0.2270 - recall: 2.4000e-04 - specificity: 1.0000 - f1_score: 4.7893e-04 - val_loss: 2.0072 - val_accuracy: 0.2524 - val_recall: 0.0063 - val_specificity: 0.9998 - val_f1_score: 0.0125
Epoch 4/30
10/10 [==============================] - 24s 2s/step - loss: 1.9690 - accuracy: 0.2760 - recall: 0.0258 - specificity: 0.9983 - f1_score: 0.0491 - val_lo

- Melakukan Prediksi ke _Data Test_ dan Menampilkan _Metric_-nya

In [ ]:
compare_train_and_test(model_for_combined_trick, normalize_x_train, y_train, normalize_x_test, y_test).transpose()

,Data Train,Data Test
Accuracy,0.539960,0.500900
Sensitivity,0.344690,0.319888
Specificity,0.985574,0.983693
F1 Score,0.463908,0.432194


3. Analisis hasil yang anda dapatkan untuk masing-masing skenario.

- Untuk skenario _feature engineering_, normalisasi nilai citra dapat meningkatkan akurasi. Ide normalisasi didasari dengan inputan awalnya bisa pada rentang $[0...255]$, hal ini dapat mengakibatkan beberapa _node_ dapat memiliki nilai yang cenderung sangat kecil dan cenderung sangat besar agar keluaran _node_-nya bisa diantara $[0...1]$. Untuk mengatasi hal itu, saya normalisasi nilainya agar berada pada rentang $[0...1]$.
- Untuk skenario menambahkan _layer_, menambahkan layer sebanyak dua dapat meningkatkan akurasi. Ide menambahkan layer sebanyak dua sebenarnya muncul karena untuk mempercepat proses eksperimen saja dan pengalaman pribadi. Semakin banyak layer, semakin baik karena dapat memodelkan ke tahap yang lebih kompleks. _Tradeoff_-nya semakin banyak layer, semakin lama eksperimen berjalan. Setelah ditambahkan sebanyak
- Untuk skenario melakukan _hyperparameter tuning_, saya hanya melakukan improvisasi pada $\text{Learning Rate}$ dan $\text{Epoch}$-nya saja. Setelah dilakukan mini skenario, _adjustment_ $\text{Learning Rate}$ dan $\text{epoch}$ saja tidak memberikan perubahan yang signifikan dan cenderung sama untuk setiap mini skenario.
- Untuk skenario gabungan ketiganya dan mengambil _hyperparameter tuning_ untuk mini skenario $1$, terlihat bahwa hasil eksperimennya mengalami peningkatan dibanding skenario-skenario yang berdiri sendiri sebelumnya.

### B. _Convolutional Neural Networks_

1. Lakukan Klasifikasi untuk dataset cifar menggunakan _Convolutional Neural Networks_ dengan arsitektur dasar sebagai berikut:
    - _Convolutional Layer_ ($25 \text{ maps}$, $\text{kernel }3 \times 3$)
    - Pooling Layer ($2 \times 2$)
    - Convolutional Layer ($50 \text{ maps}$, $\text{kernel }3 \times 3$)
    - Pooling Layer ($2 \times 2$)
    - Hidden Layer ($100 \text{ neuron}$)
    - Activation Function ($\text{ReLU function}$)
    - Output Layer ($10 \text{ class}$)
    - Activation Function ($\text{Softmax function}$)
    - Classification Result
2. Hitung metrik ($\text{accuracy}$, $\text{sensitivity}$ , $\text{specificity}$, dan $f_{1}\text{ score}$) untuk untuk data train
dan data test.

In [ ]:
convolutional_layers = []
convolutional_layers.append(Conv2D(25, kernel_size=(3, 3), activation=tf.nn.relu, padding='SAME', kernel_initializer=tf.keras.initializers.HeUniform()))
convolutional_layers.append(MaxPooling2D(2,2))
convolutional_layers.append(Conv2D(50, kernel_size=(3, 3), activation=tf.nn.relu, padding='SAME', kernel_initializer=tf.keras.initializers.HeUniform()))
convolutional_layers.append(MaxPooling2D(2,2))
convolutional_layers.append(Flatten())
convolutional_layers.append(Dense(100, activation=tf.nn.relu))
convolutional_layers.append(Dense(10, activation=tf.nn.softmax))

convolutional_model = Sequential(convolutional_layers)

## Membangun model
convolutional_model.build(input_shape=x_train.shape)
convolutional_model.output_shape

(50000, 10)

In [ ]:
convolutional_model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss='categorical_crossentropy',
    metrics=[
      'accuracy',
      recall,
      specificity,
      f1_score
    ])

In [ ]:
convolutional_model_hist = convolutional_model.fit(normalize_x_train, y_train, epochs=10, batch_size=50, validation_data=(normalize_x_test, y_test), use_multiprocessing=True)

Epoch 1/10
1000/1000 [==============================] - 73s 73ms/step - loss: 1.4219 - accuracy: 0.4938 - recall: 0.2791 - specificity: 0.9879 - f1_score: 0.3814 - val_loss: 1.1370 - val_accuracy: 0.6029 - val_recall: 0.4506 - val_specificity: 0.9830 - val_f1_score: 0.5601
Epoch 2/10
1000/1000 [==============================] - 71s 71ms/step - loss: 1.0432 - accuracy: 0.6350 - recall: 0.4916 - specificity: 0.9837 - f1_score: 0.5983 - val_loss: 1.0150 - val_accuracy: 0.6506 - val_recall: 0.5306 - val_specificity: 0.9823 - val_f1_score: 0.6267
Epoch 3/10
1000/1000 [==============================] - 71s 71ms/step - loss: 0.9204 - accuracy: 0.6800 - recall: 0.5647 - specificity: 0.9836 - f1_score: 0.6583 - val_loss: 0.9730 - val_accuracy: 0.6712 - val_recall: 0.5551 - val_specificity: 0.9812 - val_f1_score: 0.6427
Epoch 4/10
1000/1000 [==============================] - 71s 71ms/step - loss: 0.8400 - accuracy: 0.7088 - recall: 0.6096 - specificity: 0.9838 - f1_score: 0.6935 - val_loss: 0.93

In [ ]:
compare_train_and_test(convolutional_model, normalize_x_train, y_train, normalize_x_test, y_test).transpose()

,Data Train,Data Test
Accuracy,0.830500,0.690600
Sensitivity,0.780090,0.638878
Specificity,0.988639,0.975273
F1 Score,0.828022,0.685496


3. Anda dapat memperbaiki hasil yang didapat dengan melakukan 2 (dua) skenario eksperimen, yaitu menambah _layer_ dan _hyperparameter tuning_ (pada classifier).

##### Menambahkan _layer_

- Pada sesi laboratorium kali ini, saya menambahkan $\text{Convolutional Layer}$ sebanyak $2$ dengan masing-masing spesifikasi adalah:
    - $25 \text{ maps}$ dan $\text{kernel } 3 \times 3$ yang ditambahkan setelah layer pertama.
    - $50 \text{ maps}$ dan $\text{kernel } 3 \times 3$ yang ditambahkan setelah _pooling layer_ pertama.

In [ ]:
convolutional_extra_layers = []
convolutional_extra_layers.append(Conv2D(25, kernel_size=(3, 3), activation=tf.nn.relu, padding='SAME', kernel_initializer=tf.keras.initializers.HeUniform()))
convolutional_extra_layers.append(Conv2D(25, kernel_size=(3, 3), activation=tf.nn.relu, padding='SAME', kernel_initializer=tf.keras.initializers.HeUniform()))
convolutional_extra_layers.append(MaxPooling2D(2,2))
convolutional_extra_layers.append(Conv2D(50, kernel_size=(3, 3), activation=tf.nn.relu, padding='SAME', kernel_initializer=tf.keras.initializers.HeUniform()))
convolutional_extra_layers.append(Conv2D(50, kernel_size=(3, 3), activation=tf.nn.relu, padding='SAME', kernel_initializer=tf.keras.initializers.HeUniform()))
convolutional_extra_layers.append(MaxPooling2D(2,2))
convolutional_extra_layers.append(Flatten())
convolutional_extra_layers.append(Dense(200, activation=tf.nn.relu, kernel_initializer=tf.keras.initializers.HeUniform()))
convolutional_extra_layers.append(Dense(10, activation=tf.nn.softmax))

convolutional_extra_model = Sequential(convolutional_extra_layers)

## Membangun model
convolutional_extra_model.build(input_shape=x_train.shape)
convolutional_extra_model.output_shape

(50000, 10)

In [ ]:
convolutional_extra_model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss='categorical_crossentropy',
    metrics=[
      'accuracy',
      recall,
      specificity,
      f1_score
    ])

In [ ]:
convolutional_extra_model_hist = convolutional_extra_model.fit(normalize_x_train, y_train, epochs=10, batch_size=50, validation_data=(normalize_x_test, y_test), use_multiprocessing=True)

Epoch 1/10
1000/1000 [==============================] - 219s 219ms/step - loss: 1.3353 - accuracy: 0.5228 - recall: 0.3358 - specificity: 0.9856 - f1_score: 0.4385 - val_loss: 1.0289 - val_accuracy: 0.6372 - val_recall: 0.4906 - val_specificity: 0.9838 - val_f1_score: 0.5980
Epoch 2/10
1000/1000 [==============================] - 218s 218ms/step - loss: 0.9139 - accuracy: 0.6786 - recall: 0.5689 - specificity: 0.9833 - f1_score: 0.6604 - val_loss: 0.9229 - val_accuracy: 0.6750 - val_recall: 0.5656 - val_specificity: 0.9828 - val_f1_score: 0.6563
Epoch 3/10
1000/1000 [==============================] - 218s 218ms/step - loss: 0.7375 - accuracy: 0.7404 - recall: 0.6594 - specificity: 0.9847 - f1_score: 0.7327 - val_loss: 0.8408 - val_accuracy: 0.7075 - val_recall: 0.6322 - val_specificity: 0.9814 - val_f1_score: 0.7016
Epoch 4/10
1000/1000 [==============================] - 217s 217ms/step - loss: 0.5962 - accuracy: 0.7901 - recall: 0.7303 - specificity: 0.9863 - f1_score: 0.7873 - val_lo

In [ ]:
compare_train_and_test(convolutional_extra_model, normalize_x_train, y_train, normalize_x_test, y_test).transpose()

,Data Train,Data Test
Accuracy,0.977240,0.719700
Sensitivity,0.974228,0.710863
Specificity,0.997804,0.971534
F1 Score,0.977103,0.722493


##### Melakukan _Hyperparameter Tuning_

- Pada sesi laboratorium kali ini, saya mengubah optimizer yang digunakan dari $\text{Adam}$ menjadi $\text{Stochastic Gradient Descent}$

In [ ]:
convolutional_with_hyperparameter_tuning_layers = []
convolutional_with_hyperparameter_tuning_layers.append(Conv2D(25, kernel_size=(3, 3), activation=tf.nn.relu, padding='SAME', kernel_initializer=tf.keras.initializers.HeUniform()))
convolutional_with_hyperparameter_tuning_layers.append(MaxPooling2D(2,2))
convolutional_with_hyperparameter_tuning_layers.append(Conv2D(50, kernel_size=(3, 3), activation=tf.nn.relu, padding='SAME', kernel_initializer=tf.keras.initializers.HeUniform()))
convolutional_with_hyperparameter_tuning_layers.append(MaxPooling2D(2,2))
convolutional_with_hyperparameter_tuning_layers.append(Flatten())
convolutional_with_hyperparameter_tuning_layers.append(Dense(100, activation=tf.nn.relu))
convolutional_with_hyperparameter_tuning_layers.append(Dense(10, activation=tf.nn.softmax))

convolutional_with_hyperparameter_model = Sequential(convolutional_with_hyperparameter_tuning_layers)

## Membangun model
convolutional_with_hyperparameter_model.build(input_shape=x_train.shape)
convolutional_with_hyperparameter_model.output_shape

(50000, 10)

In [ ]:
convolutional_with_hyperparameter_model.compile(
    optimizer=SGD(lr=0.001, momentum=0.9),
    loss='categorical_crossentropy',
    metrics=[
      'accuracy',
      recall,
      specificity,
      f1_score
    ])

In [ ]:
convolutional_with_hyperparameter_model_hist = convolutional_with_hyperparameter_model.fit(normalize_x_train, y_train, epochs=10, batch_size=50, validation_data=(normalize_x_test, y_test), use_multiprocessing=True)

Epoch 1/10
1000/1000 [==============================] - 71s 71ms/step - loss: 1.7909 - accuracy: 0.3612 - recall: 0.0766 - specificity: 0.9960 - f1_score: 0.1296 - val_loss: 1.5731 - val_accuracy: 0.4410 - val_recall: 0.1651 - val_specificity: 0.9917 - val_f1_score: 0.2628
Epoch 2/10
1000/1000 [==============================] - 71s 71ms/step - loss: 1.4470 - accuracy: 0.4871 - recall: 0.2313 - specificity: 0.9895 - f1_score: 0.3445 - val_loss: 1.3597 - val_accuracy: 0.5224 - val_recall: 0.2854 - val_specificity: 0.9876 - val_f1_score: 0.4053
Epoch 3/10
1000/1000 [==============================] - 71s 71ms/step - loss: 1.2931 - accuracy: 0.5429 - recall: 0.3237 - specificity: 0.9872 - f1_score: 0.4473 - val_loss: 1.2638 - val_accuracy: 0.5490 - val_recall: 0.3609 - val_specificity: 0.9848 - val_f1_score: 0.4796
Epoch 4/10
1000/1000 [==============================] - 71s 71ms/step - loss: 1.1980 - accuracy: 0.5784 - recall: 0.3866 - specificity: 0.9857 - f1_score: 0.5083 - val_loss: 1.21

In [ ]:
compare_train_and_test(convolutional_with_hyperparameter_model, normalize_x_train, y_train, normalize_x_test, y_test).transpose()

,Data Train,Data Test
Accuracy,0.709100,0.657400
Sensitivity,0.578195,0.534345
Specificity,0.986873,0.982905
F1 Score,0.679644,0.629716


4. Analisis hasil yang anda dapatkan untuk masing-masing skenario.

- Untuk skenario pertama, melakukan penambahan _layer_, ternyata pada epoch ketiga sudah _convergent_. Hal ini ditunjukkan ketika kita lanjutkan _learning_-nya hingga _epoch_ ke-$10$, pada saat evaluasi perbedaan akurasi kedua model cukup jauh dan menyebabkan model sudah _overfitting_ pada bila melakukan train hingga _epoch_ ke-$10$. Hasil skenario pertama juga menunjukkan bahwa menambah layer meningkatkan akurasi model. Hal ini juga didukung dengan menambahkan layer dapat memodelkan suatu permasalahan yang lebih kompleks/spesifik.
- Untuk skenario kedua, melakukan _hyperparameter tuning_ dengan merubah _optimizer_ dari $\text{Adam}$ menjadi $\text{Stochastic Gradient Descent}$ tidak dapat meningkatkan akurasi dari model.
- Belum ditest lebih lanjut untuk merubah parameter _learning rate_ dikarenakan koneksi internet tidak memadai dan sering disconnect pada saat eksperimen menggunakan Google Collab. Sudah menggunakan laptop pribadi tetapi ternyata waktu eksperimennya memakan waktu bila dijalankan di _local_.

### C. Bandingkan hasil yang anda peroleh dari poin A dan Poin B, berikan analisis dari hasil anda

- Ambil model terbaik dari $\text{Poin A}$ adalah **metode eksperimen gabungan semua skenario**, dan model terbaik dari $\text{Poin B}$ adalah **metode eksperimen menambahkan layer**.
- Kita bisa lihat bahwa model terbaik dari $\text{Poin B}$ memiliki metric $\text{Accuracy}$ yang lebih baik dibandingkan dengan model terbaik dari $\text{Poin A}$.
- Hal menunjukkan bahwa klasifikasi citra lebih apabila menggunakan $\text{Convolutional Neural Network}$ dibandingkan $\text{Fully Connected Neural Network}$.
- Hal ini didasari dengan klasifikasi citra menggunakan $\text{Fully Connected Neural Network}$ memperlakukan input piksel masing-masing citra sebagai suatu hal yang berdiri sendiri, berbeda dengan $\text{Convolutional Neural Network}$ yang memperlakukan input piksel sebagai suatu yang berdiri dalam kelompok.